# Set up

In [ ]:
# Install Disimpy

!pip install git+https://github.com/kerkelae/disimpy.git@3595b1ebfe013ae7a4f15d84fa509dc46098b60c

In [ ]:
# Import the required packages and modules

import os
import numpy as np
import matplotlib.pyplot as plt

from disimpy import gradients, meshes, simulations, utils

In [ ]:
# Check the available GPU

!nvidia-smi

In [ ]:
# Mount Google Drive if using Google Colaboratory

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Define path and load synthetic axons

path = '/content/drive/MyDrive/Colab Notebooks/qte-model-comparison/ConFiG'
mesh = meshes.load_mesh(
    os.path.join(path, 'cell_tissue_replicated_periodic.ply')) * 1e-6
mesh_size = np.max(np.max(mesh, 0), 0)

In [ ]:
# Load simulation initial positions

n_s = int(6e5)
pos = np.zeros((n_s, 3))
for i in range(381):
    f = i + 1
    pos[int(i*1e3):int((i+1)*1e3), :] = np.loadtxt(
        os.path.join(path, 'intra_init_pos_%s.txt' % f))
pos[int(381e3)::, :] = np.loadtxt(os.path.join(path, 'extra_init_pos.txt'))

# Simulation 1

In [ ]:
# Define simulation parameters

n_s = int(6e5)
n_t = int(1e4)
diffusivity = 2e-9


# Load bvals, bvecs, and gradient waveforms

bvecs = (np.loadtxt(os.path.join(path, 'LTE-STE.bvec'))[:, 0:107]).T
bvals = np.loadtxt(os.path.join(path, 'LTE-STE.bval'))[0:107] * 1e6
ste = np.loadtxt(os.path.join(path, 'waveform_STE.txt'))[np.newaxis, :, :]
lte = np.loadtxt(os.path.join(path, 'waveform_LTE.txt'))[np.newaxis, :, :]


# Create gradient array

bvals[bvals == 0] = 1e-9
bs = np.unique(bvals)
bvals = np.concatenate([bvals, bvals])
bvecs = np.concatenate([bvecs, bvecs], axis=0)
lte_idx = np.arange(0, 107).astype(int)
ste_idx = np.arange(107, 2 * 107).astype(int)
lte_bvals = bvals[lte_idx]
ste_bvals = bvals[ste_idx]
gradient = np.concatenate(
    [lte for i in range(107)] + [ste for i in range(107)], axis=0)
T = 80e-3
dt = T / (gradient.shape[1] - 1)
Rs = [utils.vec2vec_rotmat(np.array([1, 0, 0]), i) for i in bvecs]
gradient = gradients.rotate_gradient(gradient, Rs)
gradient, dt = gradients.interpolate_gradient(gradient, dt, n_t)
gradient = gradients.set_b(gradient, dt, bvals)

In [ ]:
# Run simulation with QTE

substrate = {'type' : 'mesh',
             'mesh' : mesh,
             'initial positions' : pos,
             'periodic' : True,
             'N_sv' : 50}

signals_1 = simulations.simulation(n_s, diffusivity, gradient, dt, substrate,
                                   all_signals=True, seed=123456789)

np.save(os.path.join(path, '../Preproc-data/simulated_signals_1.npy'),
        signals_1)

# Simulation 2


In [ ]:
# Define waveforms

lte = np.loadtxt(os.path.join(path, 'waveform_LTE.txt'))
n_zeros = 8
ste = np.zeros((lte.shape[0] * 3 + n_zeros * 2, 3))
ste[0:78, 0] = lte[:, 0] 
ste[78 + n_zeros:78 + n_zeros + 78, 1] = lte[:, 0] 
ste[78 + n_zeros + 78 + n_zeros::, 2] = lte[:, 0] 
T = 80e-3 * ste.shape[0] / lte.shape[0]
n_t = int(1e4 * ste.shape[0] / lte.shape[0])
ste = ste[np.newaxis, :, :]
lte = np.copy(ste)
lte[:, :, 1::] = 0


# Create gradient array

gradient = np.concatenate(
    [lte for i in range(107)] + [ste for i in range(107)], axis=0)
dt = T / (gradient.shape[1] - 1)
Rs = [utils.vec2vec_rotmat(np.array([1, 0, 0]), i) for i in bvecs]
gradient = gradients.rotate_gradient(gradient, Rs)
gradient, dt = gradients.interpolate_gradient(gradient, dt, n_t)
gradient = gradients.set_b(gradient, dt, bvals)

In [ ]:
# Run simulation with TDE1

signals_2 = simulations.simulation(n_s, diffusivity, gradient, dt, substrate,
                                   all_signals=True, seed=123456789)

np.save(os.path.join(path, '../Preproc-data/simulated_ignals_2.npy'),
        signals_2)

# Simulation 3

In [ ]:
# Define waveforms

ste = np.zeros((504, 3))
ste[1:3, 0] = 1
ste[101:103, 0] = -1
ste[201:203, 1] = 1
ste[301:303, 1] = -1
ste[401:403, 2] = 1
ste[501:503, 2] = -1
ste = ste[np.newaxis, :, :]
lte = np.copy(ste)
lte[:, :, 1::] = 0
T = 503e-3
n_t = int(1e4 * 503 / 80)


# Create gradient array

gradient = np.concatenate(
    [lte for i in range(107)] + [ste for i in range(107)], axis=0)
dt = T / (gradient.shape[1] - 1)
Rs = [utils.vec2vec_rotmat(np.array([1, 0, 0]), i) for i in bvecs]
gradient = gradients.rotate_gradient(gradient, Rs)
gradient, dt = gradients.interpolate_gradient(gradient, dt, n_t)
gradient = gradients.set_b(gradient, dt, bvals)

In [ ]:
# Run simulation with TDE2

signals_3 = simulations.simulation(n_s, diffusivity, gradient, dt, substrate,
                                   all_signals=True, seed=123456789)

np.save(os.path.join(path, '../Preproc-data/simulated_signals_3.npy'),
        signals_3)